In [1]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# 数据集路径
train_dir = 'Pin/train'
validation_dir = 'Pin/test'

# 设置参数
input_size = 224
batch_size = 32
num_epochs = 10 
num_classes = 2
bidirectional = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 数据预处理
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomRotation(20),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
    ]),
    'validation': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomRotation(20),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
    ]),
}

image_datasets = {x: torchvision.datasets.ImageFolder(train_dir if x == 'train' else validation_dir, data_transforms[x]) for x in ['train', 'validation']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'validation']}

classes = image_datasets['train'].classes
classes_index = image_datasets['train'].class_to_idx
print(classes)
print(classes_index)


# 使用VGG16提取特征
vgg16 = models.vgg16(pretrained=True)
for param in vgg16.parameters():
    param.requires_grad = False

vgg16.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=10000, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=100000, out_features=5000, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=5000, out_features=1000, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=1000, out_features=100, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=100, out_features=2, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
)

vgg16.to(device)

# 添加LSTM层
class VGG16_LSTM(nn.Module):
    def __init__(self, base_model):
        super(VGG16_LSTM, self).__init__()
        self.base_model = base_model
        self.lstm = nn.LSTM(input_size=2, hidden_size=1, num_layers=2, batch_first=True)
        self.fc = nn.Linear(1 * (2 if bidirectional else 1), num_classes, bias=True)

    def forward(self, x):
        x = self.base_model(x)
        x, _ = self.lstm(x.view(x.size(0), 1, -1))
        x = self.fc(x[:, -1, :])
        return x

model = VGG16_LSTM(vgg16).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# 训练模型
for epoch in range(num_epochs):
    print("epoch:", epoch)
    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
            
torch.save(model.state_dict(), 'morris_md2_use_lstm_32.pth')

['md2', 'morris']
{'md2': 0, 'morris': 1}


/Library/Python/3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Python/3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch: 0
train Loss: 0.8630 Acc: 0.1031
validation Loss: 0.8399 Acc: 0.1595
Epoch 0/9
----------
epoch: 1
train Loss: 0.8601 Acc: 0.1031
validation Loss: 0.8374 Acc: 0.1595
Epoch 1/9
----------
epoch: 2
train Loss: 0.8571 Acc: 0.1031
validation Loss: 0.8349 Acc: 0.1595
Epoch 2/9
----------
epoch: 3
train Loss: 0.8543 Acc: 0.1031
validation Loss: 0.8324 Acc: 0.1595
Epoch 3/9
----------
epoch: 4
train Loss: 0.8514 Acc: 0.1031
validation Loss: 0.8298 Acc: 0.1595
Epoch 4/9
----------
epoch: 5
train Loss: 0.8486 Acc: 0.1031
validation Loss: 0.8273 Acc: 0.1595
Epoch 5/9
----------
epoch: 6
train Loss: 0.8458 Acc: 0.1031
validation Loss: 0.8248 Acc: 0.1595
Epoch 6/9
----------
epoch: 7
train Loss: 0.8430 Acc: 0.1031
validation Loss: 0.8224 Acc: 0.1595
Epoch 7/9
----------
epoch: 8
train Loss: 0.8402 Acc: 0.1031
validation Loss: 0.8199 Acc: 0.1595
Epoch 8/9
----------
epoch: 9
train Loss: 0.8374 Acc: 0.1031
validation Loss: 0.8175 Acc: 0.1595
Epoch 9/9
----------
